## Compare Non-Optimized Circuit and Optimized Circuit

The algorithm used in this toturial is from [QuantumFlow](https://www.nature.com/articles/s41467-020-20729-5) (Box-2 on Page 10). 

In [1]:
try:
    import torch  
    print('Module torch was installed')
except ImportError:    
    print("Installinng torch 1.8.1")
    !pip install -q torch==1.8.1
    import qiskit  

try:
    import torchvision  
    print('Module torchvision was installed')
except ImportError:    
    print("Installinng torchvision 0.4.0")
    !pip install -q torchvision==0.4.0
    import qiskit  

try:
    import qiskit  
    print('Module qiskit was installed')
except ImportError:    
    print("Installinng qiskit 0.14.0")
    !pip install -q qiskit==0.14.0
    import qiskit  


import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image


import sys
import functools
from qiskit import  QuantumCircuit, ClassicalRegister
from qiskit import Aer, execute
import math

from circuit.lib_qiskit import *
import qiskit

from qiskit import IBMQ

# IBMQ.save_account('2d7fb4a2d08cbb349872fe2229939996c6b01d6e420a129f9a63a5c8bb7c615394c422b44240dc09c275e506c6bad13d80776785cbfb36ef2e6ac297c9b85715')
# IBMQ.load_account()



print = functools.partial(print, flush=True)

################ Zhirui on 12-30-2020 ################
# Parameters of settings
######################################################
interest_num = [3,6]
ori_img_size = 28
img_size = 4
num_workers = 0 # number of subprocesses to use for data loading
batch_size = 1# how many samples per batch to load
inference_batch_size = 1
data_path = '/home/hzr/Software/quantum/qc_mnist/pytorch/data'
isppd = False

Module torch was installed
Module torchvision was installed
Module qiskit was installed


In [2]:
from training.lib_dataloader import to_quantum_matrix,load_data

################ Zhirui on 12-30-2020 ################
# load data.
######################################################

train_loader, test_loader = load_data(interest_num,data_path,isppd,img_size,batch_size,inference_batch_size,False)
for batch_idx, (data, target) in enumerate(test_loader):
    torch.set_printoptions(threshold=sys.maxsize)
    print("Batch Id: {}, Target: {}".format(batch_idx,target))
    quantum_matrix = to_quantum_matrix(data)
    break

Batch Id: 0, Target: tensor([1])


/home/hzr/anaconda3/envs/qf/lib/python3.8/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448216815/work/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
################ Weiwen on 12-30-2020 ################
# Parameters of the trained model
# The training procedure will be found in another repo
# https://github.com/weiwenjiang/QuantumFlow
######################################################

# Model initialization

resume_path = '/home/hzr/Software/quantum/qc_mnist/model/exe_mnist.py_2021_08_08-16_00_34/model_best.tar'
checkpoint = torch.load(resume_path, map_location='cpu')
print(checkpoint['state_dict']['fc0.theta'])
print(checkpoint['state_dict']['fc1.weight'])
theta = checkpoint['state_dict']['fc0.theta']
weight = checkpoint['state_dict']['fc1.weight']

# print(checkpoint[])
# print(checkpoint['fc1.weight'])


tensor([-5.3796,  0.7002,  1.8451, -5.7278,  0.1500, -0.3549,  3.5649,  5.7828],
       dtype=torch.float64)
tensor([[-37.2579,  37.5035, -36.9454,  36.6320, -37.5422, -30.4944,  45.8007,
           9.1910,  39.2518,  36.8431,  40.8417,  34.0796,  -7.6042, -39.2305,
          34.1263,  43.2855],
        [-35.8045, -12.2471, -31.9292, -22.6312,  19.5908, -40.1494,  35.5673,
          41.5370, -32.6166, -36.5595, -15.5469, -37.7452, -36.7410, -37.6752,
          38.4902,  39.5903]])


In [4]:
from training.lib_qf import binarize
circuit = QuantumCircuit()
u_mat_c = UMatrixCircuit(16,2)
inps = u_mat_c.add_input_qubits(circuit)
u_mat_c.forward(circuit,inps,quantum_matrix)

In [5]:
################ Weiwen on 12-30-2020 ################
# Generate the circuit of v-layer
######################################################

vqc = VQuantumCircuit(4,2)
vqc.forward(circuit,inps,'v10',np.array(theta))




In [6]:
################ Weiwen on 12-30-2020 ################
# Generate the circuit of u-layer
######################################################
#define your input and output number
u_layer = FFNNCircuit(16,2) 
#init circuit
# circuit = QuantumCircuit()
#add input qubit to your circuit
# inps = u_layer.add_in_qubits(circuit)

#add auxiliary qubit to your circuit
aux =u_layer.add_aux(circuit)

#add output qubit to your circuit
u_layer_out_qubits = u_layer.add_out_qubits(circuit)
#add ulayer to your circuit
u_layer.forward(circuit,binarize(weight),inps,u_layer_out_qubits,aux)


circuit.barrier()
c_reg = ClassicalRegister(2,"reg")
circuit.add_register(c_reg)
circuit.measure(u_layer_out_qubits[0],c_reg[0])
circuit.measure(u_layer_out_qubits[1],c_reg[1])

#show your circuit
circuit.draw('mpl',fold=50)



AttributeError: type object 'FFNNCircuit' has no attribute 'n_class'

### Non-Optimized Circuit v.s. Optimized Circuit

Let's test and compare!


In [ ]:
################ Weiwen on 12-30-2020 ################
# Quantum simulation
######################################################

# Optimized one
qc_shots=8192
opt_counts = fire_ibmq(circuit,qc_shots,True)
print("="*10,"Optimized Circuit","="*10)
(opt_mycount,bits) = analyze(opt_counts)
opt_class_prob=[]
for b in range(bits):
    opt_class_prob.append(float(opt_mycount[b])/qc_shots)


print("="*10,"Optimized Circuit","="*10)
print("Optimized Circuit Depth:",circuit.depth())
print("Result of optimized QC:",opt_class_prob)
print("Prediction class: {}".format(opt_class_prob.index(max(opt_class_prob))))
print("Target class: {}".format(target[0]))
if opt_class_prob.index(max(opt_class_prob))==target[0]:
    print("Correct prediction")
else:
    print("Incorrect prediction")
print("="*30)